SqlAlchemy helps to proceed logic of data manipulation 
when we handle interaction with user



We have three main points


![Illustration](.excalidraw.png)






In [24]:
from reprlib import Repr as _Repr

set(getattr(_Repr, '__repr_blacklist__', set()))

set()

In [30]:
from typing import Optional, List
from sqlalchemy import ForeignKey
from sqlalchemy.orm import Mapped, mapped_column,declarative_base
from sqlalchemy.orm import relationship, backref
from datetime import datetime
from sqlalchemy_repr import RepresentableBase

Base = declarative_base(cls=RepresentableBase)



class User(Base):
    __tablename__ = "user"

    id: Mapped[int]= mapped_column(primary_key=True)
    name: Mapped[str] 
    mipt_mail: Mapped[Optional[str]] = mapped_column(default='')
    auth: Mapped[Optional[bool]]  = mapped_column(default=False)
    admin: Mapped[Optional[bool]] = mapped_column(default=False)
    homeworks: Mapped[List["UserHomeworkResult"]] = relationship(backref=backref('user'),cascade='all, delete-orphan')
    entrance_dttm: Mapped[datetime] = mapped_column(default=datetime.now())
    valid_from_dttm: Mapped[datetime] = mapped_column(default=datetime.now(),onupdate=datetime.now)

    # def __repr__(self):
    #     return pprint(self.__dict__) 

class Homework(Base):
    __tablename__ = 'homework'

    id: Mapped[int] = mapped_column(primary_key = True) 
    week: Mapped[int]
    description: Mapped[Optional[str]]
    homework_link: Mapped[Optional[str]]
    valid_from_dttm: Mapped[datetime] = mapped_column(default=datetime.now(),onupdate=datetime.now)

    # def __repr__(self):
    #     return (
    #         f'Homework id: {self.id}'
    #         f'Week: {self.week}'
    #         f'Description: {self.description}'
    #         f'Homework Link: {self.homework_link}'
    #         f'Valid_from_dttm: {self.valid_from_dttm}'
    #     )


class UserHomeworkResult(Base):
    __tablename__ = 'homework_results'
 
    id: Mapped[int] = mapped_column(primary_key = True) 
    homework_id : Mapped[int] = mapped_column(ForeignKey("homework.id")) 
    user_id: Mapped[int] = mapped_column(ForeignKey("user.id")) 
    student_homework_link: Mapped[Optional[str]]
    result: Mapped[int] = mapped_column(default=0)
    valid_from_dttm: Mapped[datetime] = mapped_column(default=datetime.now(),onupdate=datetime.now)
    homework: Mapped["Homework"] = relationship(backref='results')


    # def __repr__(self):
    #     return (
    #         f'Homework id: {self.homework_id}'
    #         f'User id: {self.user_id}'
    #         f'student_homework_link: {self.student_homework_link}'
    #         f'result: {self.result}'
    #         f'Valid_from_dttm: {self.valid_from_dttm}'
    #         f'Homework :{self.homework}'
    #     )


In [31]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# база будет создана в памяти
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)


In [32]:
from sqlalchemy.orm import Session

# add_homework

with Session(engine) as session:
    hw1 = Homework(
        week = 1,
        description = 'Работа с SQlAlchemy',
        homework_link = 'github.com'
    )
    hw2 = Homework(
        week = 2,
        description = 'Продвинутая работа с SQlAlchemy',
        homework_link = 'github2.com'
    )

    session.add_all([hw1,hw2])
    
    session.commit()

In [33]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Homework)
    for ent in session.execute(stmt).scalars():
        print(ent)

<Homework id=1, week=1, description='Работа с SQlAlchemy', homework_link='github.com', valid_from_dttm='2023-09-26T22:34:52.395110'>
<Homework id=2, week=2, description='Продвинутая работа с SQlAlchemy', homework_link='github2.com', valid_from_dttm='2023-09-26T22:34:52.395110'>


In [34]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Homework)
    homeworks = session.execute(stmt).scalars()
    new_user = User(id=1,name='Nikita')
    new_user.homeworks = [UserHomeworkResult(homework=hw) for hw in homeworks] 
    session.merge(new_user)
    session.commit()

/tmp/ipykernel_153562/209486939.py:7: SAWarning: Object of type <UserHomeworkResult> not in session, add operation along 'Homework.results' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  session.merge(new_user)


In [45]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Homework)
    homeworks = session.execute(stmt).scalars()
    new_user = User(id=1,name='Misha12')
    session.merge(new_user)
    session.commit()

In [46]:
with Session(engine) as session:
    stmt = select(User).where(User.id == 1)
    user: User = session.execute(stmt).scalars().first()
    print(user)

<User id=1, name='Misha12', mipt_mail='', auth=False, admin=False, entrance_dttm='2023-09-26T22:34:52.392912', valid_from_dttm='2023-09-26T22:36:02.187492'>


In [8]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(User).where(User.id == 2)
    user: User = session.execute(stmt).scalars().first()
    print(user)
    for hw in user.homeworks:
        print(hw)
    user.homeworks = [user.homeworks[0]]
    session.merge(user)
    session.commit()

Telegram id: 1Name: NikitaMipt_mail: 
Homework id: 1User id: 1student_homework_link: Noneresult: 0Valid_from_dttm: 2023-09-24 21:27:37.212283Homework :Homework id: 1Week: 1Description: Работа с SQlAlchemyHomework Link: github.com
Homework id: 2User id: 1student_homework_link: Noneresult: 0Valid_from_dttm: 2023-09-24 21:27:37.212283Homework :Homework id: 2Week: 2Description: Продвинутая работа с SQlAlchemyHomework Link: github2.com


In [12]:
with Session(engine) as session:
    stmt = select(User).where(User.id == 1)
    user: User = session.execute(stmt).scalars().first()
    print(user.homeworks)

[Homework id: 1User id: 1student_homework_link: Noneresult: 0Valid_from_dttm: 2023-09-24 21:27:37.212283Homework :Homework id: 1Week: 1Description: Работа с SQlAlchemyHomework Link: github.com]
